In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import math

## Loading IrisDataset and Labelling it

In [2]:
iris = load_iris()
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [3]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d
def label(val, *boundaries):
    if val < boundaries[0]:
        return 'a'
    elif val < boundaries[1]:
        return 'b'
    elif val < boundaries[2]:
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, feat):
    second = df[feat].mean()
    mini = df[feat].min()
    maxi = df[feat].max()
    first = (second + mini)/2
    third = (second + maxi)/2
    return df[feat].apply(label, args = (first, second, third))

In [4]:
df['sl_label'] = toLabel(df, 'sl')
df['sw_label'] = toLabel(df, 'sw')
df['pl_label'] = toLabel(df, 'pl')
df['pw_label'] = toLabel(df, 'pw')
# to drop the previous and extra columns of no use from now
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

## TreeNode class for Storing Output of Node and Feature on which it Splits

In [5]:
class TreeNode:
    def __init__(self, data,output):
        # data represents the feature upon which the node was split when fitting the training data
        # data = None for leaf node
        self.data = data
        # children of a node are stored as a dicticionary with key being the value of feature upon which the node was split
        # and the corresponding value stores the child TreeNode
        self.children = {}
        # output represents the class with current majority at this instance of the decision tree
        self.output = output
        # index will be used to assign a unique index to each node
        self.index = -1
        
    def add_child(self,feature_value,obj):
        self.children[feature_value] = obj

## Our Decision Tree Classifier

In [6]:
class DecisionTreeClassifier:
    def __init__(self):
        # root represents the root node of the decision tree built after fitting the training data
        self.__root = None
               
    # calculating entropy
    def entropy(self, Y):
        # store the total entropy
        epy = 0.0 
        d = {} # stores the count of all possible values in Y
        for i in Y:
            if i in d:
                d[i] = d[i] + 1
            else:
                d[i] = 1
        total = len(Y)
        for i in d:
            p = d[i]/total
            epy = epy + (((-1.0) * p)*math.log2(p))
        return epy

    # calculating the gain ratio
    def gain(self, X, Y, feat):
        info_original = self.entropy(Y)  # current entropy at the noce
        info_f = 0.0                  # it tells the entropy after splitting upon the selected feature
        original_size = X.shape[0]
        unique_values = set(X[feat])
        X[X.shape[1]] = Y
        split_info = 0.0
        for i in unique_values:
            new_X = X[X[feat] == i]
            newX = new_X.iloc[:,:-1]
            newY = new_X[new_X.shape[1]-1]
            curr_size = new_X.shape[0]
            p = curr_size/original_size
            info_f = info_f + (1.0 * p * self.entropy(newY))
            split_info = split_info - (1.0 * p * math.log2(p))
        info_gain = info_original - info_f
        if split_info == 0 :
            return math.inf
        gain_ratio = info_gain / split_info
        return gain_ratio    
    
    def decisionTree(self, Xtrain, Ytrain, unused_features, level):
        # returns the root of the Decision Tree(which consists of TreeNodes) built after fitting the training data
        # Here Nodes are printed as in PREORDER traversl
        # level represents depth of the tree
        # We split a node on a particular feature only once (in a given root to leaf node path)
        
        # storing the count of the unique values in Ytrain 
        arr = np.array(Ytrain)
        lis = arr.ravel()
        uni = {}
        for i in range(len(lis)):
            if lis[i] not in uni:
                uni[lis[i]] = 1
            else:
                uni[lis[i]] = uni[lis[i]] + 1

        # output will store the maxing count from each of the possible values in Y
                
        # Base Case 1
        # pure node
        if len(uni) == 1:
            output = None
            print("Level ", level)
            print("Pure Node")
            print("Current Entropy is 0.0")
            if 0 in uni:
                output = 0
                print("Count of 0 is", uni[0])
            elif 1 in uni:
                output = 1
                print("Count of 1 is", uni[1])
            else:
                output = 2
                print("Count of 2 is", uni[2])
            print()
            return TreeNode(None, output)

        # Base Case 2
        # length of features is zero
        if(len(unused_features) == 0):
            print("Level ", level)
            output = None
            max_count = -1000
            if 0 in uni:
                if max_count < uni[0]:
                    max_count = uni[0]
                    output = 0
                print("Count of 0 is", uni[0])
            else:
                print("Count of 0 is", 0)
            if 1 in uni:
                if max_count < uni[1]:
                    max_count = uni[1]
                    output = 1
                print("Count of 1 is", uni[1])
            else:
                print("Count of 1 is", 0)
            if 2 in uni:
                if max_count < uni[2]:
                    max_count = uni[2]
                    output = 2
                print("Count of 2 is", uni[2])
            else:
                print("Count of 2 is", 0)
            print("Current Entropy is ", self.entropy(Ytrain))
            print("Reached Leaf Node")
            print()
            return TreeNode(None, output)

        maxgain = -math.inf    # to store the max gain till then
        best_feature = None    # to store the best feature
        for f in unused_features:
            g = self.gain(Xtrain, Ytrain, f)
            if(g > maxgain):
                maxgain = g
                best_feature = f
                
        # printing the current Node and calculating output to store in NODE
        print("Level ", level)
        print(unused_features)
        output = None
        max_count = -1000
        if 0 in uni:
            if max_count < uni[0]:
                max_count = uni[0]
                output = 0
            print("Count of 0 is", uni[0])
        else:
            print("Count of 0 is", 0)
        if 1 in uni:
            if max_count < uni[1]:
                max_count = uni[1]
                output = 1
            print("Count of 1 is", uni[1])
        else:
            print("Count of 1 is", 0)
        if 2 in uni:
            if max_count < uni[2]:
                max_count = uni[2]
                output = 2
            print("Count of 2 is", uni[2])
        else:
            print("Count of 2 is", 0)
        print("Current Entropy is ", self.entropy(Ytrain))
        print("Split on Feature ",best_feature," with Gain Ratio ",maxgain)
        print()

        # creating current node with output
        current_node = TreeNode(best_feature,output)

        unique_values = set(Xtrain[best_feature])
        unused_features.remove(best_feature)
        level = level + 1
        Xtrain[Xtrain.shape[1]] = Ytrain
        
        # recursively calling on the splits and storing each child of current node in it
        for i in unique_values:
            new_Z = Xtrain[Xtrain[best_feature] == i]
            X = new_Z.iloc[:,:-1]
            Y = new_Z[new_Z.shape[1] - 1]
            node = self.decisionTree(X, Y, unused_features, level)
            current_node.add_child(i,node)

        unused_features.add(best_feature)
        return current_node
    
    # fitting and starting our Decision Tree
    def fit(self, X, Y, ft):
        self.root = self.decisionTree(X, Y, ft, 0)
    
    # predicting output for each row
    def predIn(self, dat, node):
        # if the node is leaf node
        if len(node.children) == 0 :
            return node.output

        val = dat[node.data] # represents the value of feature on which the split was made       
        if val not in node.children :
            return node.output

        # Recursively call on the splits
        return self.predIn(dat, node.children[val])

    # predicting the outputs of X and returning them
    def pred(self, X):
        y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            y[i] = self.predIn(X.iloc[i,:], self.root)
        return y

    # to calculate the score and return the predictions and score
    def score(self, X, Y):
        Y_pred = self.pred(X)
        count = 0      # count will store the number of truth predictions
        for i in range(len(X)):
            if Y_pred[i] == Y[i]:
                count = count + 1
        return Y_pred, count/len(X)

## Fitting Data and Printing the each Node in Different Levels

In [7]:
clf1 = DecisionTreeClassifier()     # creating object of our Decision Tree
y = pd.DataFrame(iris.target)       # y contains the target values of our training data
unused_features = set(df.columns)   # unused_features contains the features names of our trainig data
raw = df.copy()                     # saving a copy of Data to work again on it
yraw = y.copy()                     # saving a copy of Data to work again on it
clf1.fit(raw,yraw,unused_features)  # fitting data to our Decision Tree

Level  0
{'pw_label', 'pl_label', 'sl_label', 'sw_label'}
Count of 0 is 50
Count of 1 is 50
Count of 2 is 50
Current Entropy is  0.04819212460330587
Split on Feature  pw_label  with Gain Ratio  -0.15183643117217138

Level  1
Pure Node
Current Entropy is 0.0
Count of 2 is 34

Level  1
{'pl_label', 'sl_label', 'sw_label'}
Count of 0 is 0
Count of 1 is 40
Count of 2 is 16
Current Entropy is  0.863120568566631
Split on Feature  pl_label  with Gain Ratio  0.4334099495621067

Level  2
Pure Node
Current Entropy is 0.0
Count of 2 is 8

Level  2
{'sl_label', 'sw_label'}
Count of 0 is 0
Count of 1 is 39
Count of 2 is 8
Current Entropy is  0.6581912658132185
Split on Feature  sl_label  with Gain Ratio  0.12674503775809332

Level  3
Pure Node
Current Entropy is 0.0
Count of 1 is 2

Level  3
{'sw_label'}
Count of 0 is 0
Count of 1 is 23
Count of 2 is 7
Current Entropy is  0.783776947484701
Split on Feature  sw_label  with Gain Ratio  0.07092036405148876

Level  4
Pure Node
Current Entropy is 0.0
Co

## Printing Predictions and Score of our Decision Tree

In [8]:
# converting the 2D dataFrame to 1D array
arr = np.array(y)                   
lis = arr.ravel()

# checking our score and predictions
our_predictions, our_score = clf1.score(df, lis)
print("Predictions", our_predictions)              # predictions of our Decision Tree
print("Score :", our_score)                    # score of our Decision Tree

Predictions [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 2 2 2
 2 1]
Score : 0.9533333333333334


## Fitting the Data in inbulit Decision Tree and printing Predictions and Score

In [9]:
# Using the inbuilt sklearn Decision tree and comparing it with our model on the same dataset
import sklearn.tree
clf3 = sklearn.tree.DecisionTreeClassifier()

# to convert the strings a, b, c, d to integer values
for i in range(len(raw)):
    for j in range(len(raw.iloc[i,:])):
        z = raw.iloc[i,j]
        o = None
        if z == 'a':
            o = 1
        elif z == 'a':
            o = 2
        elif z == 'c':
            o = 3
        else:
            o = 4
        raw.iloc[i,j] = o


clf3.fit(raw, yraw)                  # fit the data to inbulit tree
Y_pred3 = clf3.predict(raw)         # predicting the results from inbuilt tree
print("Predictions", Y_pred3)        # printing the predictions from inbuilt tree
sklearn_score = clf3.score(raw, yraw)  
print("Score :", sklearn_score)      # score of the inbuilt tree

Predictions [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1
 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 1 2 2 2 2 2 1 2 2 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 2 1 2
 2 1]
Score : 0.92
